<a href="https://colab.research.google.com/github/bruno-orozco/mk/blob/master/API_Banco_de_M%C3%A9xico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import datetime
import plotly.graph_objects as go

#Variables iniciales
hoy = datetime.datetime.today().strftime('%Y-%m-%d')
token = '4d8d8d3dc2480b3bb624bae16cc566174ddca5d39fe1cf3887be73103ae61d9c'
serie = "SR16643"
fechainicio = "1900-01-01"
fechafin = hoy

def bmx_datos(serie, fechainicio, fechafin, token):
    """
    Parameters
    ----------
    serie : string
        Sustituye la serie por la de tu interés.
        Obtén el catálogo de series en: 
        https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries#
    fechainicio : string
        Obtener la fecha inicial en formato '%Y-%m-%d'
    fechafin : 
        Obtener la fecha final en formato '%Y-%m-%d'
    token : string
        Obtener token de autenticación.

    Returns
    -------
    datframe
        Retorna un dataframe de los datos obtenidos.
    """
    #Llamado al API
    api_banxico = "https://www.banxico.org.mx/SieAPIRest/service/v1/series"

    url = "{}/{}/datos/{}/{}".format(api_banxico, serie, fechainicio, fechafin)
    
    headers = {'Bmx-Token' : token}
    response = requests.get(url, headers=headers)

    status = response.status_code
    
    if status != 200:
        return print("Error {}".format(status))
        
    raw_data = response.json()
    
    data = raw_data['bmx']['series'][0]['datos']
    
    df = pd.DataFrame(data)

    df["fecha"] = pd.to_datetime(df['fecha'],
                                 format="%d/%m/%Y")
    
    df["dato"] = df["dato"].replace({',':''}, regex=True)
    
    df["dato"] = df["dato"].astype(float) 

    
    return df

In [ ]:
def pib_precios_correintes(df):
    """
    Función ejemplo para obtener gráfica con el DF obtenido
    """



    fig = go.Figure([go.Scatter(x=df['fecha'], y=df['dato'],
                                mode='lines+markers',
                                fill="tonexty",
                                )])
    fig.update_layout(title="Producto Interno Brunto, a precios de mercado (México)",
                   xaxis_title='Fecha',
                   yaxis_title='Millones de pesos (precios correintes)')

    fig.show()

In [ ]:

df = bmx_datos(serie, fechainicio, fechafin, token)
pib_precios_correintes(df)
df